In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@stretaildataproj.dfs.core.windows.net/products")

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
select * from products


In [0]:
%sql
CREATE OR REPLACE FUNCTION retail_databricks_catalog.bronze.discount_function(para_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN para_price * 0.90 

In [0]:
%sql
SELECT product_id,
       price,
       retail_databricks_catalog.bronze.discount_function(price) AS discounted_price
FROM products;


In [0]:
df = df.withColumn("discounted_price" , expr("retail_databricks_catalog.bronze.discount_function(price)"))
df.display()

In [0]:
%sql
create or replace function retail_databricks_catalog.bronze.upper_function(para_brand String)
returns string
language python
as
$$
    return para_brand.upper()
$$


In [0]:
%sql
select * ,  retail_databricks_catalog.bronze.upper_function(brand) as brand_upper from products

###  Data Writing To Silver Layer In The Products Folder

In [0]:
df.write.mode("append").format("delta").save("abfss://silver@stretaildataproj.dfs.core.windows.net/products/")

In [0]:

%sql
CREATE TABLE IF NOT EXISTS retail_databricks_catalog.silver_schema.products_silver
USING DELTA
LOCATION 'abfss://silver@stretaildataproj.dfs.core.windows.net/products/'

In [0]:
%sql
select * from retail_databricks_catalog.silver_schema.products_silver